In [1]:
# test_order.py
# -*- coding: utf-8 -*-

import sys, json, time, random

# =============== 集中配置（按你的 runner 风格） ===============
random.seed(time.time())

# 项目路径
SIMWORLD_DIR      = r"D:\LLMDelivery-LJ\SimWorld"
LLM_DELIVERY_DIR  = r"D:\LLMDelivery-LJ\LLM-Delivery"

# 数据文件
ROADS_JSON        = r"D:\LLMDelivery-LJ\Test_Data\test\roads.json"
WORLD_JSON        = r"D:\LLMDelivery-LJ\Test_Data\test\progen_world_enriched.json"

# 订单池大小
ORDER_POOL_SIZE   = 10
# ===========================================================

# 项目路径注入
sys.path.insert(0, SIMWORLD_DIR)
sys.path.insert(0, LLM_DELIVERY_DIR)

from Base.Map import Map
from Base.Order import OrderManager  # 需使用你最新的 Base/Order.py（init 即 _pick_meta 绑定）

# ----------------- 小工具 -----------------
def _load_world_nodes(path: str):
    with open(path, "r", encoding="utf-8") as f:
        obj = json.load(f)
    return obj.get("nodes", [])

def _order_to_dict(o) -> dict:
    dist_m = float(o.distance_cm) / 100.0
    return {
        "id": o.id,
        "pickup": {"x": float(o.pickup_address.x), "y": float(o.pickup_address.y)},
        "dropoff": {"x": float(o.delivery_address.x), "y": float(o.delivery_address.y)},
        "pickup_road_name": getattr(o, "pickup_road_name", ""),
        "dropoff_road_name": getattr(o, "dropoff_road_name", ""),
        "road_name": getattr(o, "road_name", ""),
        "accepted": bool(o.is_accepted),
        "delivered": bool(o.has_delivered),
        "distance_cm": float(o.distance_cm),
        "distance_m": dist_m,
        "distance_km": dist_m / 1000.0,
        "eta_s": float(o.eta_s),
        "time_limit_s": float(o.time_limit_s),
        "earnings": float(o.earnings),
        "path_len": len(o.path_nodes),
        "items": list(o.items),
        "note": o.special_note,
    }

# ----------------- 主流程 -----------------
def main():
    # 1) 构建地图
    m = Map()
    m.import_roads(ROADS_JSON)
    m.import_pois(WORLD_JSON)

    # 2) 世界节点（只从里头抽；抽不到/路由失败会抛异常）
    world_nodes = _load_world_nodes(WORLD_JSON)

    # 3) 生成订单池并填满
    om = OrderManager(capacity=ORDER_POOL_SIZE)
    om.fill_pool(m, world_nodes)

    # 4) 一键文本输出（包含 pickup/dropoff 的 road_name）
    print("=" * 60)
    print(om.orders_text())

    # 5) JSON 汇总（可选）
    json_summary = [_order_to_dict(o) for o in om.list_orders()]
    print("\nJSON summary:")
    print(json.dumps(json_summary, ensure_ascii=False, indent=2))

if __name__ == "__main__":
    main()


c:\Users\Shadow\miniconda3\envs\citynav\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Orders (10):
[Order #0]
  Pickup : (-13027.6, -4205.8)  | road: 2nd road (left)
  Dropoff: (-56120.7, -9294.7) | road: 14th road (right)
  Path   : 28 nodes
  Dist   : 548.7 m  (0.549 km)
  ETA    : 7 min
  Limit  : 8 min
  $$     : $6.15
  Special Note: 
------------------------------------------------------------
[Order #1]
  Pickup : (6581.4, 4072.2)  | road: 1st road (left)
  Dropoff: (-24419.2, -4572.3) | road: 5th road (right)
  Path   : 19 nodes
  Dist   : 393.6 m  (0.394 km)
  ETA    : 5 min
  Limit  : 6 min
  $$     : $3.37
  Special Note: 
------------------------------------------------------------
[Order #2]
  Pickup : (-24378.2, -10996.5)  | road: 5th road (right)
  Dropoff: (-36120.7, -35994.2) | road: 13th road (left)
  Path   : 23 nodes
  Dist   : 436.0 m  (0.436 km)
  ETA    : 6 min
  Limit  : 8 min
  $$     : $5.35
  Special Note: 
------------------------------------------------------------
[Order #3]
  Pickup : (6581.4, 4072.2)  | road: 1st road (left)
  Dropoff: (-